In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy.stats import gaussian_kde
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor  
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error,f1_score
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import GradientBoostingClassifier

### Область работы 2 (поиск  модели .... )

In [3]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

train = 'train.csv'
df = pd.read_csv(train)
df.drop_duplicates(inplace = True)
df # содержит только имя файла, без имен папок !!!

In [ ]:
X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)

In [ ]:
cat_enc_1 = ['C6_enc','C5_enc','C4_enc','C12_enc']
num_features = ['N2','N3','N7','N10','N13','N14']
cat_features = ['C1','C9','C11']

# проверим метод КНН

In [ ]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [ ]:
model_baseline =Pipeline(steps=[
        ('preproc', ct),
        ('estimator', KNeighborsClassifier())
    ],
)
model_baseline.fit(X_train, y_train)

In [ ]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
   'estimator__n_neighbors': [30, 35, 40, 45, 50],
    'estimator__p': [2,3,4],
    'estimator__weights': ['uniform', 'distance']  
}

In [ ]:
baseline_grid = GridSearchCV(model_baseline, n_jobs=-1, param_grid=param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

In [ ]:
print("Лучшие параметры:", baseline_grid.best_params_)

In [ ]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

# Проверим метод градиентного бустинга

In [ ]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [ ]:
model_baseline_1 = Pipeline(steps=[
        ('preproc', ct),
        ('estimator', GradientBoostingClassifier())
    ],
)
model_baseline_1.fit(X_train, y_train)

In [ ]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
  # 'estimator__learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7],
  #  'estimator__subsample': [0.3,0.4,0.5,0.6],
  #  'estimator__n_estimators': [300,400,500],
    'estimator__criterion':['friedman_mse','squared_error'],
    'estimator__max_depth': [2,3,4,5]
}

In [ ]:
baseline_grid = GridSearchCV(model_baseline_1, n_jobs=-1, param_grid = param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

### найдем лучшие параметры, а в дальнейшем некоторые подберем вручную

In [ ]:
print("Лучшие параметры:", baseline_grid.best_params_)

In [ ]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

In [ ]:
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', StandardScaler())    
 ])
gb = Pipeline( steps = [   
('preproc', ct),
('estimator', GradientBoostingClassifier(criterion = 'friedman_mse',
                                         n_estimators = 400,subsample=0.1,learning_rate=0.01,
                                         max_depth =3 ,random_state=42))])
gb.fit(X_train, y_train)

In [ ]:
y_pred = gb.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

In [ ]:
y_pred = gb.predict(X_train)
f1 = f1_score(y_train, y_pred)
f1

In [ ]:
clf = GradientBoostingClassifier(learning_rate = 0.1, subsample = 0.5, random_state = 42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Расчет метрики F1
f1 = f1_score(y_test, y_pred)
f1

In [ ]:
np.mean(cross_val_score(clf, X, y, scoring = 'f1', cv = 100))

# Рандом форест

In [ ]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [ ]:
model_baseline_2 = Pipeline(steps=[
        ('preproc', ct),
        ('estimator', RandomForestClassifier())
    ],
)
model_baseline_1.fit(X_train, y_train)

In [ ]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
   'estimator__criterion': ['gini','entropy'],
    #'estimator__n_estimators': range(50,400),
    'estimator__max_features': range(4,11),
    #'estimator__min_samples_split': [2, 5, 10],
    
}

In [ ]:
baseline_grid = GridSearchCV(model_baseline_2, n_jobs=-1, param_grid = param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

In [ ]:
print("Лучшие параметры:", baseline_grid.best_params_)

In [ ]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

### Соберем модель и снова добавим несколько параметров вручную

In [ ]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', MinMaxScaler())    
 ])
gb = Pipeline( steps = [   
('preproc', ct),
('estimator', RandomForestClassifier(criterion = 'gini', max_features = 6,
                                    n_estimators = 100,
                                    random_state=42))])
gb.fit(X_train, y_train)

In [ ]:
forest = RandomForestClassifier(criterion= 'entropy', n_estimators = 59, max_features = 9,
                                random_state = 42)
forest.fit(X_train, y_train)
y_pred_best = forest.predict(X_test)
print('Правильность на тестовом наборе:', f1_score(y_test, y_pred_best))

In [ ]:
y_pred = gb.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

In [ ]:
y_pred = gb.predict(X_train)
f1 = f1_score(y_train, y_pred)
f1

###### Лучшее значение = 